In [1]:
import os
import sys
from collections import OrderedDict

In [2]:
sys.path.append("../..")

In [3]:
import spatialpy
spatialpy.__file__

'../../spatialpy/__init__.py'

# Model

In [4]:
import matplotlib.pyplot as plt
import numpy

# Global Constants
MAX_X_DIM = 5.0
MIN_X_DIM = -5.0
TOL = 1e-9

In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
class Edge1(spatialpy.Geometry):
    def inside(self, x, on_boundary):
        return abs(x[0] - MAX_X_DIM) < 0.05
class Edge2(spatialpy.Geometry):
    def inside(self, x, on_boundary):
        return abs(x[0] - MIN_X_DIM) < 0.05
class Middle(spatialpy.Geometry):
    def inside(self, x, on_boundary):
        return abs(x[0] - MIN_X_DIM) >= 0.05

In [7]:
class cylinderDemo3D(spatialpy.Model):
    def __init__(self, model_name="cylinder_demo3d"):
        spatialpy.Model.__init__(self, model_name)

        self.timestep_size = 1
        
        # System constants
        D_const = 0.1

        # Define Species
        A = spatialpy.Species(name="A", diffusion_constant=D_const)
        B = spatialpy.Species(name="B", diffusion_constant=D_const)
        self.add_species([A, B])

        # Define Geometry
        self.mesh = spatialpy.Mesh.read_xml_mesh('cylinder.xml')

        # Define Subdomains
        self.set_type(Middle(), 1)
        self.set_type(Edge1(),  2)
        self.set_type(Edge2(),  3)
        
        # Restrict the movement of Chemical Species
        self.restrict(A,[1,2])
        self.restrict(B,[1,3])

        vol = self.mesh.get_vol()
        print("vol",vol)
        type = self.mesh.type
        left = numpy.sum(vol[type == 2])
        right = numpy.sum(vol[type == 3])
        print("left "+str(left)+" right "+str(right))
        
        k_react = spatialpy.Parameter(name="k_react", expression=1.0)
        k_creat1 = spatialpy.Parameter(name="k_creat1", 
                                     expression=100/left)
        k_creat2 = spatialpy.Parameter(name="k_creat2", 
                                     expression=100/right)
        self.add_parameter([k_react, k_creat1,k_creat2])


        # Define Reactions
        R1 = spatialpy.Reaction(reactants=None, products={A:1}, 
                                rate=k_creat1, restrict_to=2)
        R2 = spatialpy.Reaction(reactants=None, products={B:1}, 
                              rate=k_creat2, restrict_to=3)
        R3 = spatialpy.Reaction(reactants={A:1, B:1}, products=None, 
                              rate=k_react)
        self.add_reaction([R1, R2, R3])
        
        #self.add_initial_condition(spatialpy.ScatterInitialCondition(A,10000,[1]))


        # Define simulation timespan
        #self.set_timesteps(1, 200)
        self.timespan(range(500))

In [8]:
model = cylinderDemo3D()

vol [0.01513526 0.07034112 0.02382667 ... 0.01674217 0.02120607 0.01969156]
left 0.5092013833059308 right 0.505804729089437


# Linked List

In [9]:
class Node():
    n_prev = None
    n_next = None
    
    def __init__(self, data=None):
        self.data = data

In [10]:
class LinkedList():
    
    def __init__(self):
        self.head = Node()
        self.tail = Node()
        self.count = 0
        self.head.n_next = self.tail
        self.tail.n_prev = self.head
        
    def __str__(self):
        pnt_str = [f"Head: {self.head.data}", f"Tail: {self.tail.data}",
                   f"Count: {self.count}", f"Data:", "*"*80]
        data = []
        node = self.head
        while node is not None:
            data.append(str(node.data))
            node = node.n_next
        pnt_str.extend([f"[{', '.join(data)}]", "*"*80])
        return '\n'.join(pnt_str)
        
    def add_node(self, data, round_to=0):
        if round_to:
            data=round(data, round_to)
        
        if self.head.data is None:
            self.head.data = data
        elif self.tail.data is None:
            self.tail.data = data
        else:
            node = Node(data)
            node.n_prev = self.tail
            self.tail.n_next = node
            self.tail = node
            
        self.count += 1
        
    def add_all_nodes(self, data, round_to=0):
        if not isinstance(data, list):
            raise ValueError("Data must be a list")
            
        for value in data:
            self.add_node(value, round_to=round_to)
            
    def is_sorted(self, verbose=0):
        
        errors = []
        node = self.head.n_next
        while node is not None:
            if node.n_prev.data > node.data:
                error = f"Error: {node.n_prev.data}, \"{node.data}\""
                errors.append(error)
            node = node.n_next
                
        if errors:
            print("The linked list is not sorted or has not been sorted incorrectly")
            print(f"{len(errors)} errors found")
        else:
            print("The linked list was sorted correctly")
            
        if verbose:
            print("*"*80)
            print("\n".join(errors))
            
    def sort(self, method="quicksort", old=True):
        if method == "quicksort":
            if old:
                self.__quicksort_old(self.head, self.tail)
            else:
                self.__quicksort_new(self.head, self.tail)
        else:
            print(f"The {method} is not currently supported")
            
    def __quicksort_old(self, min_n, max_n):
        if min_n is None or max_n is None or min_n == max_n or max_n.n_next == min_n:
            return
        
        pivot = self.__quicksort_partition_avg(min_n, max_n)
        self.__quicksort_old(min_n, pivot.n_prev)
        self.__quicksort_old(pivot.n_next, max_n)
        
    def __quicksort_new(self, min_n, max_n):
        if min_n is None or max_n is None or min_n == max_n or max_n.n_next == min_n:
            return
        
        pivot = self.__quicksort_partition_avg(min_n, max_n)
        self.__quicksort_new(min_n, pivot.n_prev)
        self.__quicksort_new(pivot, max_n)
        
    def __quicksort_partition_avg(self, min_n, max_n):
        partition_value = (min_n.data + max_n.data) / 2.0
        left = min_n
        right = max_n
        
        while left != right and right.n_next != left:
            # 
            while left != right and left.data <= partition_value:
                left = left.n_next
            # 
            while left != right and right.data > partition_value:
                right = right.n_prev
                
            if left != right and right.n_next != left:
                self.__sort_swap(left, right)
        return right
    
    def __sort_swap(self, a, b):
        tmp = b.data
        b.data = a.data
        a.data = tmp

## Get data from mesh for linked list

In [11]:
data = list(map(lambda coords: coords[0], model.mesh.vertices))
# data = data[::50]
len(data)

1059

## Quicksort using the original sort function (frozen pivot)

In [12]:
ll = LinkedList()
ll.add_all_nodes(data, round_to=3)
ll.is_sorted()
# print(ll)

The linked list is not sorted or has not been sorted incorrectly
472 errors found


In [13]:
%time ll.sort()

CPU times: user 3.98 ms, sys: 160 µs, total: 4.14 ms
Wall time: 4.37 ms


In [14]:
ll.is_sorted(verbose=1)

The linked list is not sorted or has not been sorted incorrectly
275 errors found
********************************************************************************
Error: -4.741, "-4.786"
Error: -4.72, "-4.743"
Error: -4.722, "-4.725"
Error: -4.71, "-4.72"
Error: -4.673, "-4.7"
Error: -4.693, "-4.695"
Error: -4.639, "-4.652"
Error: -4.625, "-4.629"
Error: -3.98, "-4.597"
Error: -4.581, "-4.591"
Error: -4.577, "-4.589"
Error: -4.575, "-4.581"
Error: -4.528, "-4.56"
Error: -4.308, "-4.47"
Error: -4.383, "-4.401"
Error: -4.163, "-4.374"
Error: -4.334, "-4.34"
Error: -4.319, "-4.327"
Error: -4.23, "-4.287"
Error: -4.209, "-4.245"
Error: -4.188, "-4.244"
Error: -4.227, "-4.234"
Error: -4.165, "-4.204"
Error: -4.163, "-4.166"
Error: -3.698, "-4.135"
Error: -4.091, "-4.122"
Error: -3.914, "-3.941"
Error: -3.894, "-3.898"
Error: -3.841, "-3.865"
Error: -3.841, "-3.852"
Error: -3.671, "-3.786"
Error: -3.775, "-3.784"
Error: -3.755, "-3.772"
Error: -3.741, "-3.742"
Error: -3.677, "-3.723"
Error: 

In [15]:
print(ll)

Head: -5.0
Tail: 5.0
Count: 1059
Data:
********************************************************************************
[-5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -4.741, -4.786, -4.72, -4.743, -4.722, -4.725, -4.722, -4.71, -4.72, -4.673, -4.7, -4.698, -4.698, -4.693, -4.695, -4.664, -4.662, -4.639, -4.652, -4.647, -4.636, -4.625, -4.629, -4.628, -4.625, -3.98, -4.597, -4.596, -4.581, -4.591, -4.577, -4.589, -4.575, -4.581, -4.581, -4.575, -4.528, -4.56, -4.556, -4.549, -4.536, -4.488, -4.488, -4.308, -4.47, -4.447, -4.429, -4.422, -4.414, -4.383, -4.401, -4.163, -4.374, -4.363, -4.334, -4.34, -4.34, -4.319, -4.327, -4.321, -4.23, -4.287, -4.282, -4.257, -4.254, -4.253, -4.209, -4.245, -4.188, -4.244, -4.227, -4.234, -4.165, -4.204, -4.163, -4.166, -3.698, -4.135, -4.126, -4.123, -4.091, -4

## Quicksort using the fixed sort function (pivot included in 2nd call)

In [16]:
ll2 = LinkedList()
ll2.add_all_nodes(data, round_to=3)
ll2.is_sorted()
# print(ll2)

The linked list is not sorted or has not been sorted incorrectly
472 errors found


In [17]:
%time ll2.sort(old=False)

CPU times: user 4.72 ms, sys: 109 µs, total: 4.83 ms
Wall time: 5.59 ms


In [18]:
ll2.is_sorted()

The linked list was sorted correctly


In [19]:
print(ll2)

Head: -5.0
Tail: 5.0
Count: 1059
Data:
********************************************************************************
[-5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -5.0, -4.786, -4.743, -4.741, -4.725, -4.722, -4.722, -4.72, -4.72, -4.71, -4.7, -4.698, -4.698, -4.695, -4.693, -4.673, -4.664, -4.662, -4.652, -4.647, -4.639, -4.636, -4.629, -4.628, -4.625, -4.625, -4.597, -4.596, -4.591, -4.589, -4.581, -4.581, -4.581, -4.577, -4.575, -4.575, -4.56, -4.556, -4.549, -4.536, -4.528, -4.488, -4.488, -4.47, -4.447, -4.429, -4.422, -4.414, -4.401, -4.383, -4.374, -4.363, -4.34, -4.34, -4.334, -4.327, -4.321, -4.319, -4.308, -4.287, -4.282, -4.257, -4.254, -4.253, -4.245, -4.244, -4.234, -4.23, -4.227, -4.209, -4.204, -4.188, -4.166, -4.165, -4.163, -4.163, -4.135, -4.126, -4.123, -4.122, -4.121, -4.091, -